*Берутся видео с датасета https://disk.yandex.ru/d/_vjY_E84Bs1p-Q, бьются на кадры, с помощью yolov8n в боксы выделяются люди, затем (вручную) в csv файл записываются метки кадров*

*Тут получаем кадры с видео*

In [39]:
counter = 0

In [40]:
import cv2
import os
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
from PIL import Image



video_path = 'Shoplifting3.mp4' #одно из видео с диска
output_folder = 'output_frames' #куда сохранять

cap = cv2.VideoCapture(video_path)
frame_count = 0

counter += 1

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    frame_name = f'frame_{counter}_{frame_count}.jpg'
    cv2.imwrite(os.path.join(output_folder, frame_name), frame)

cap.release()
cv2.destroyAllWindows()

/home/anastasia/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [5]:
import pandas as pd
df_frames = pd.DataFrame(columns = ['file_name', 'labels'])

In [7]:
df_frames

,file_name,labels


*Добавляем в датафрейм новые кадры*

In [41]:
import os
image_dir = 'output_frames'
image_files = os.listdir(image_dir)
new_rows = []
for image_file in image_files:
        file_name = os.path.basename(image_file)
        if file_name not in df_frames['file_name'].values:
            new_row = {'file_name': file_name, 'labels': 0}
            new_rows.append({'file_name': file_name, 'labels': 0})
df_frames = pd.concat([df_frames, pd.DataFrame(new_rows)], ignore_index=True)

In [42]:
df_frames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1086 entries, 0 to 1085
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  1086 non-null   object
 1   labels     1086 non-null   object
dtypes: object(2)
memory usage: 17.1+ KB


In [35]:
def set_label(name):
    df_frames.loc[df_frames['file_name'] == name, 'labels'] = 1

In [58]:
set_label('frame_142_1.jpg') 

In [59]:
df_frames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1086 entries, 0 to 1085
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  1086 non-null   object
 1   labels     1086 non-null   object
dtypes: object(2)
memory usage: 17.1+ KB


*Код на случай, если надо будет получать yolo боксы*

In [9]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
from PIL import Image
model = YOLO('yolov8n.pt')
model.classes = [0]
image_path = 'output_frames/frame_1.jpg'
save_directory = ''
results = model(source = image_path, classes=0, show = False, imgsz=640, conf=0.3, iou=0.4, save = True, save_dir=save_directory)


image 1/1 /home/anastasia/Desktop/project_testing/output_frames/frame_1.jpg: 480x640 3 persons, 83.0ms
Speed: 4.4ms preprocess, 83.0ms inference, 6.2ms postprocess per image at shape (1, 3, 480, 640)
Results saved to /home/anastasia/runs/detect/predict4


In [ ]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
from PIL import Image

def crop_image(image, box):
    x_min, y_min, x_max, y_max = box
    width, height = image.size
    box_width = x_max - x_min
    box_height = y_max - y_min
    x_min_2 = max(0, x_min - box_width/3)
    y_min_2 = max(0, y_min - box_height/3)
    x_max_2 = min(width, x_max + box_width/3)
    y_max_2 = min(height, y_max + box_height/3)
    area = (x_min_2, y_min_2, x_max_2, y_max_2)
    cropped_img = image.crop(area)
    return cropped_img
    

def recieve_box(image):
    model = YOLO('yolov8n.pt')
    model.classes = [0]
    results = model(source = image, classes=0, show = False, imgsz=640, conf=0.3, iou=0.4, save = False)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    cropped_images = []
    for box in boxes:
        cropped_images.append(crop_image(image, box))
    return cropped_images

model = YOLO('yolov8n.pt')
model.classes = [0]
image_path = '/content/drive/MyDrive/yolo_dataset/test/images/cam-pos-video-five_0mABuXZe_mp4-36_jpg.rf.9f17c4aa766ded211ed3a70d1728403f.jpg'
results = model(source = image_path, classes=0, show = False, imgsz=640, conf=0.3, iou=0.4, save = True)

import os
image_path = '/content/drive/MyDrive/yolo_dataset/test/images/cam-pos-video-five_0mABuXZe_mp4-36_jpg.rf.9f17c4aa766ded211ed3a70d1728403f.jpg'
boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
save_dir = 'runs/detect/predict18'
for box in boxes:
    x_min, y_min, x_max, y_max = box
    #print(f"Box coordinates: ({x_min}, {y_min}), ({x_max}, {y_max})")
    img = Image.open(image_path)
    width, height = img.size
    box_width = x_max - x_min
    box_height = y_max - y_min
    x_min_2 = max(0, x_min - box_width/3)
    y_min_2 = max(0, y_min - box_height/3)
    x_max_2 = min(width, x_max+ box_width/3)
    y_max_2 = min(height, y_max + box_height/3)
    area = (x_min_2, y_min_2, x_max_2, y_max_2)
    cropped_img = img.crop(area)
    save_path = os.path.join(save_dir, "cropped_.jpg")
    cropped_img.save(save_path)